## Sistema Agente de Reportero de Noticias Usando el SDK de Semantic Kernel y el Servicio de Agentes de Azure AI

In [ ]:
# Instalando la versión específica de la biblioteca de Semantic Kernel.
%pip install semantic-kernel==1.28.1

In [ ]:
# Instalando la versión específica de la biblioteca para el servicio de Agentes de Azure AI.
%pip install azure-ai-projects==1.0.0b7

#### Importando bibliotecas y utilidades importantes

In [1]:
from typing import Annotated
from semantic_kernel import Kernel # El orquestador principal de Semantic Kernel.
import os # Para interactuar con el sistema operativo (leer variables de entorno).
import asyncio # Para ejecutar operaciones de forma asíncrona.
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion # El conector para los modelos de chat de Azure OpenAI.
from dotenv import load_dotenv # Para cargar variables desde un archivo .env.
from semantic_kernel.planners import SequentialPlanner # El planificador que creará el plan de acción.
from typing import Annotated # Para añadir descripciones a los parámetros de las funciones.
from semantic_kernel.functions.kernel_function_decorator import kernel_function # El decorador para convertir funciones de Python en herramientas para la IA.
from azure.ai.projects import AIProjectClient # El cliente para el servicio de Agentes de Azure AI.
from azure.identity import DefaultAzureCredential # Para la autenticación con Azure.
from azure.ai.projects.models import BingGroundingTool # La herramienta específica para la búsqueda con Bing.

#### Cargando las variables de entorno

In [2]:
# Carga las configuraciones desde el archivo .env para mantener las claves seguras.
load_dotenv()
# Lee cada una de las variables necesarias del entorno.
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_deployment_name = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL")
ai_project_connection_string = os.getenv("AI_PROJECT_CONNECTION_STRING")
bing_connection_name = os.getenv("BING_CONNECTION_NAME")

#### Creando un cliente de Azure AI Project para conectar con el Servicio de Agentes de Azure AI

In [3]:
# Este objeto se usará para crear los agentes "trabajadores" o "especialistas".
project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(), # Usa las credenciales de Azure para autenticarse.
        conn_str=ai_project_connection_string # Apunta al servicio de Agentes correcto usando la cadena de conexión.
        )

#### Creando una clase de Plugin "Agents" que incluirá los plugins nativos para el Kernel

In [4]:
# Esta clase agrupa todas las funciones que actuarán como agentes especialistas.
class Agents:
    # --- Agente Especialista 1: El Investigador Web ---
    @kernel_function(
        # La descripción es VITAL. El Planificador la lee para entender qué hace esta herramienta.
        description="Esta función se usará para utilizar un agente de IA de Azure con capacidad de búsqueda web usando la API de Búsqueda de Bing",
        # El nombre es cómo el Planificador se referirá a esta herramienta en el plan.
        name="WebSearchAgent"
    )
    def web_search_agent(
        self, # Parámetro estándar en los métodos de una clase.
        # 'Annotated' permite describir cada parámetro. Esto también ayuda al Planificador a saber qué información pasarle.
        query: Annotated[str, "La consulta del usuario para la cual se necesita obtener información contextual de la web"]
    ) -> Annotated[str, "La respuesta del agente de búsqueda web"]:
        # Obtiene los detalles de la conexión de Bing preconfigurada en Azure.
        bing_connection = project_client.connections.get(connection_name=bing_connection_name)
        conn_id = bing_connection.id
        # Prepara la herramienta de Bing para dársela al agente.
        bing = BingGroundingTool(connection_id=conn_id)
        
        # Crea un agente de Azure AI temporal y especializado. Su única misión es buscar en la web.
        agent = project_client.agents.create_agent(
            model=azure_openai_deployment_name,
            name="bing-assistant",
            instructions="Eres un asistente útil",
            tools=bing.definitions, # Se le asigna la herramienta de búsqueda.
            headers={"x-ms-enable-preview": "true"},
        )
        
        # Se crea un hilo de conversación para el agente.
        thread = project_client.agents.create_thread()
        
        # Se añade la consulta de búsqueda como el primer mensaje en la conversación.
        message = project_client.agents.create_message(
                thread_id=thread.id,
                role="user",
                content=query,
            )
        
        # Se le ordena al agente que procese la conversación y genere una respuesta.
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        
        # Se recupera la lista de mensajes, donde la respuesta del agente será la más reciente.
        messages = project_client.agents.list_messages(thread_id=thread.id)
        
        # Se imprime la respuesta para poder ver el progreso en la consola.
        print("Respuesta del agente de búsqueda web:")
        print("\n")
        print(messages.data[0].content[0].text.value)
        
        # Se devuelve únicamente el texto de la respuesta, que será la entrada para el siguiente paso del plan.
        return messages.data[0].content[0].text.value
    
    # --- Agente Especialista 2: El Guionista de Noticias ---
    @kernel_function(
       description="Esta función usará un agente de IA de Azure para preparar un guion para un reportero de noticias basado en información reciente para un tema específico",
       name="NewsReporterAgent"
   )
    def news_reporter_agent(
        self,
        topic: Annotated[str, "El tema para el cual se ha obtenido la información/noticia más reciente"],
        latest_news: Annotated[str,"La información más reciente para un tema específico"]
    ) -> Annotated[str, "La respuesta del NewsReporterAgent, que es el guion para el reportero"]:

        # Crea un segundo agente de Azure AI, también temporal y especializado. Su única misión es escribir.
        agent = project_client.agents.create_agent(
            model=azure_openai_deployment_name,
            name="news-reporter",
            # Las instrucciones son muy detalladas para asegurar que el resultado sea de alta calidad.
            instructions="""Eres un asistente útil destinado a preparar un guion para un reportero de noticias basado en la información más reciente sobre un tema específico, los cuales se te proporcionarán.
                El canal de noticias se llama MSinghTV y el reportero se llama John. Se te dará el tema y la información más reciente para ese tema. Prepara un guion para el reportero John basado en esta información.""",
            headers={"x-ms-enable-preview": "true"}, # No tiene herramientas, solo instrucciones.
        )
        
        # Al igual que el otro agente, necesita su propio hilo de conversación.
        thread = project_client.agents.create_thread()
        
        # Se le entrega la información (el tema y las noticias) en forma de un mensaje.
        message = project_client.agents.create_message(
                thread_id=thread.id,
                role="user",
                content=f"""El tema es {topic} y la información más reciente es {latest_news}""",
            )
        
        # Se le ordena al agente que escriba el guion.
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        
        # Se recupera el guion final de la conversación.
        messages = project_client.agents.list_messages(thread_id=thread.id)
        
        # Se imprime el guion final.
        print("Guion para el reportero de noticias:")
        print("\n")
        print(messages.data[0].content[0].text.value)
        
        # Se devuelve el guion como el resultado final de esta función.
        return messages.data[0].content[0].text.value

#### Creando el Kernel del SDK de Semantic Kernel (El "Cerebro")

In [5]:
# Se crea la instancia principal del Kernel, el orquestador.
kernel = Kernel()

# Se define un alias para nuestro servicio de IA.
service_id = "default"

# Se conecta el Kernel al modelo de lenguaje de Azure OpenAI. Ahora el Kernel tiene "poder de cómputo".
kernel.add_service(
    AzureChatCompletion(service_id=service_id,
                        api_key=azure_openai_key,
                        deployment_name=azure_openai_deployment_name,
                        endpoint = azure_openai_endpoint
    )
)

#### Creando Nuestro Planificador (El "Director")

In [6]:
# Se crea la instancia del Planificador. Este será nuestro 'Mánager de Proyectos'.
planner = SequentialPlanner(
    kernel, # Se le da acceso al Kernel para que pueda usar sus funciones y su IA.
    service_id # Se le indica qué servicio de IA debe usar para 'pensar' y crear el plan.
)

#### Añadiendo los Plugins de Agentes como plugins nativos a nuestro Kernel

In [7]:
# Se añade la clase 'Agents' como un plugin. Este es el paso donde el 'Director' conoce a su 'equipo de especialistas'.
agents_plugin = kernel.add_plugin(Agents(), "Agents")

#### Estableciendo la consulta del usuario (El Objetivo del CEO)

In [8]:
# Esta es la instrucción de alto nivel que le damos al sistema.
objetivo = f"preparar un guion de noticias para John sobre las últimas noticias del mundo?"

#### Invocando al Planificador e imprimiendo sus pasos/pensamientos

In [9]:
# Se define una función asíncrona para la fase de planificación.
async def call_planner():
    # Aquí es donde el planificador 'piensa' y crea el plan.
    return await planner.create_plan(objetivo)

# Se ejecuta la planificación. 'await' se usa aquí porque un notebook ya tiene un bucle de eventos asíncronos activo.
sequential_plan = await call_planner()

# Se imprime el plan resultante para ver qué ha decidido hacer la IA.
print("Los pasos del plan son:")
for step in sequential_plan._steps:
    print(
        f"- {step.description.replace('.', '') if step.description else 'No description'} usando {step.metadata.fully_qualified_name} con parámetros: {step.parameters}"
    )

Los pasos del plan son:
- Esta función se usará para utilizar un agente de IA de Azure con capacidad de búsqueda web usando la API de Búsqueda de Bing usando Agents-WebSearchAgent con parámetros: {'query': 'últimas noticias del mundo'}
- Esta función usará un agente de IA de Azure para preparar un guion para un reportero de noticias basado en información reciente para un tema específico usando Agents-NewsReporterAgent con parámetros: {'topic': 'mundo', 'latest_news': '$LATEST_WORLD_NEWS'}


#### Ejecutando el "Plan" generado por el Planificador

In [10]:
# Se define una función asíncrona para la fase de ejecución.
async def generate_answer():
    # Aquí se le ordena al Kernel que ejecute cada paso del plan en secuencia.
    return await sequential_plan.invoke(kernel)

# Se ejecuta el plan completo.
result = await generate_answer()

# Se imprime el resultado final del último paso del plan.
print(result)


Respuesta del agente de búsqueda web:


Lo siento, no pude obtener las últimas noticias del mundo debido a un problema técnico con el acceso a la información en línea. Te recomendaría que revises fuentes de noticias confiables como BBC, CNN, El País o Reuters para obtener la información más reciente.
Guion para el reportero de noticias:


**Guion para el Reportero John en MSinghTV**

---

**[Intro Música de Noticiero]**

**John:** 

"¡Buenas tardes! Les damos la bienvenida a una nueva edición de MSinghTV. Soy John, y hoy les traemos un resumen de las noticias más recientes alrededor del mundo. Lamentablemente, debido a algunos problemas técnicos, no hemos podido acceder a las últimas noticias en línea, pero no se preocupen, ya estamos trabajando para resolverlo."

**[Pausa breve y cambio de cámara]**

**John:** 

"Por esa razón, queremos recordarle a nuestra audiencia algunas de las fuentes más confiables donde pueden mantenerse informados. Recomendamos consultar medios como BBC, CNN, 